In [ ]:
## Database 사용

# SQLite3
# 기기 내부에서만 사용할 수 있는 DBMS
# 주로 모바일이나 소형 기기에서 관계형 데이터베이스를 생성해 주는 소프트웨어
# 생성된 DB는 외부에서 접근을 허용하지 않음
# 파이썬은 SQLite3를 기본으로 제공하고 있어 제약 없이 import하여 사용 가능

# https://sqlitebrowser.org/dl/ 에서 DB Browser 설치하기

## 테이블 생성
# CREATE table [if not exists] 테이블명 (
#       컬럼명 데이터타입 제약조건, ...
# )

## 데이터타입
# TEXT : 문자형         ex) TEXT(size)
# NUMERIC : 숫자형      ex) NUMERIC(size)
# INTIGER : 정수형
# REAL : 실수형
# BLOB : 대용량 자료

## 제약조건
# PRIMARY KEY : 기본키
# UNIQUE : 중복 불가
# NOT NULL : 생략 불가
# CHECK : 제약조건
# DEFAULT : 기본값

## CRUD -> Create(Insert), Read(Select), Update(Update), Delete(Delete)

In [37]:
# SQLite3 사용 예시

# 모듈 import
import sqlite3
print(sqlite3.sqlite_version_info)      # 버전 정보 : 3.37.2 - 잘 설치되어 있음을 확인!

try :
    # DB 연동
    conn = sqlite3.connect('data/sqlite_db')    # DB생성 및 연결객체 생성
    
    # SQL 실행 객체 cursor 생성
    cursor = conn.cursor()      # DB와 연결하여 실행할 객체 생성 - 쿼리 내용 처리
    
    # table 생성
    sql = 'create table if not exists test_table \
        (name text(10), phone text(13), addr text(50))'
    cursor.execute(sql)
    
    # 레코드 추가
    # cursor.execute("insert into test_table VALUES ('주세인','010-2610-2291','서울시 목동')")
    # cursor.execute("insert into test_table VALUES ('허웅','010-5092-2740','하남시 망월동')")
    # cursor.execute("insert into test_table VALUES ('주다롱','010-2525-5252','해씨별')")
    # cursor.execute("insert into test_table VALUES ('주호롱','010-8253-8253','서울시 목동 케이지')")
    # conn.commit()   # DB에 반영

    # 레코드 조회
    cursor.execute("SELECT * FROM test_table")
    rows = cursor.fetchall()        # list 내의 튜플로 데이터 반환
    
    # 출력 1
    for row in rows :
        print(row)
    
    # 출력 2
    print('이름 \t 전화번호 \t 주소')
    for row in rows :
        print(row[0],'\t',row[1],'\t',row[2])
        
except Exception as e :
    print('DB 연동 에러 :',e)
    conn.rollback()
finally :
    cursor.close()
    conn.close()

(3, 37, 2)
('주세인', '010-2610-2291', '서울시 목동')
('허웅', '010-5092-2740', '하남시 망월동')
('주다롱', '010-2525-5252', '해씨별')
('주호롱', '010-8253-8253', '서울시 목동 케이지')
('아무개', '010-0000-0000', '성남시')
이름 	 전화번호 	 주소
주세인 	 010-2610-2291 	 서울시 목동
허웅 	 010-5092-2740 	 하남시 망월동
주다롱 	 010-2525-5252 	 해씨별
주호롱 	 010-8253-8253 	 서울시 목동 케이지
아무개 	 010-0000-0000 	 성남시


In [ ]:
# [실습]
import sqlite3

try :
    # DB 연동
    conn = sqlite3.connect('data/sqlite_db')    # DB생성 및 연결객체 생성
    
    # SQL 실행 객체 cursor 생성
    cursor = conn.cursor()      # DB와 연결하여 실행할 객체 생성 - 쿼리 내용 처리
        
    # 메뉴를 사용하여 다음 코드를 작성 (1. 출력, 2. 수정, 3. 삭제, 0. 종료)
    # name으로 입력값을 받아서 정보를 수정하는 코드 작성하세요
    while True :
        sel = int(input('1. 출력, 2. 수정, 3. 삭제, 0. 종료'))
        if sel == 1 :
            cursor.execute('SELECT * FROM test_table')
            rows = cursor.fetchall()
            for row in rows :
                print('이름 :',row[0])
                print('번호 :',row[1])
                print('주소 :',row[2])
        elif sel == 2 :
            name = input('수정할 이름을 입력하세요 : ')
            s = cursor.execute(f"SELECT * FROM test_table where name ='{name}'").fetchall()
            print(s)
            num = int(input('수정할 타입을 입력하세요(1.이름, 2.번호, 3.주소) : '))
            data = input('수정할 데이터를 입력하세요 : ')
            if num == 1 : col = 'name'
            elif num == 2 : col = 'phone'
            elif num == 3 : col = 'addr'
            sql = f"UPDATE test_table SET {col} = '{data}' WHERE name = '{name}' ;"
            cursor.execute(sql)
            conn.commit()
        elif sel == 3 :
            name = input('삭제할 이름을 입력하세요 : ')
            cursor.execute(f"DELETE FROM test_table WHERE name = '{name}' ;")
            conn.commit()
        elif sel == 0 :
            print('데이터 수정 종료 !')
            break
    
except Exception as e :
    print('DB 연동 에러 :',e)
    conn.rollback()
finally :
    cursor.close()
    conn.close()

In [ ]:
# [실습] pickle 저장 후 DB에 입력 후 꺼내서 시각화하기
import urllib; import pickle; from bs4 import BeautifulSoup; from data.crawling import crawling; import sqlite3
word_count = {}

for i in range(1,8) :
    url = 'https://news.naver.com/main/list.naver?mode=LS2D&sid2=248&sid1=103&mid=shm&date=20220401&page='+str(i)
    word_count = crawling(url, word_count)

try :
    conn = sqlite3.connect('data/weather_db')
    cursor = conn.cursor()
    # sql = "CREATE table if not exists weather (num INTEGER PRIMARY KEY AUTOINCREMENT, word TEXT(20), frequency INTEGER)"
    # for x, y in word_count.items() :
        # sql = f"INSERT INTO weather(word, frequency) VALUES ('{x}',{y})"
        # cursor.execute(sql)
    # conn.commit()
    
    sql = "SELECT word, frequency FROM weather ORDER BY frequency DESC;"
    lists = cursor.execute(sql).fetchmany(5)
    print(lists)
except Exception as e :
    print('예외 발생 -> ',e)
    conn.rollback()
finally :
    cursor.close()
    conn.close()

In [ ]:
# # weather 테이블 비우기 구문
# import sqlite3

# try :
#     conn = sqlite3.connect('data/weather_db')
#     cursor = conn.cursor()
       
#     sql = "DELETE FROM weather"     # SQLite는 따로 truncate문을 가지지 않으므로, 조건절 없이 delete를 사용하면 모든 데이터가 삭제됨
#     cursor.execute(sql)
#     conn.commit()
# except Exception as e :
#     print('예외 발생 -> ',e)
#     conn.rollback()
# finally :
#     cursor.close()
#     conn.close()